<a href="https://colab.research.google.com/github/Sushmitha-Katti/EVA-4/blob/master/Session12/S12_V3(Reduce_Overfitting)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object classification on Tiny Imagenet Data

In [1]:
!pip install --no-cache-dir git+https://github.com/albu/albumentations > /dev/null && echo "Albumentation library is successfully installed!"
!git clone https://github.com/Sushmitha-Katti/PyTNet 
import sys
sys.path.append("PyTNet")

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-cwgu5xsy
Albumentation library is successfully installed!
fatal: destination path 'PyTNet' already exists and is not an empty directory.


## **Importing Required files**

In [0]:
import torch
import train_test_loader
import show_images
import train_test
import evaluate
from Albumentationtransform import AlbumentationTransforms
from GradCam import GradCAM,visualize_cam
from LR_Range_Test import LR_test
from tinyimagenet import TinyImageNetDataSet

In [3]:
import albumentations as A
import cv2
channel_means = (0.5, 0.5, 0.5)
channel_stdevs = (0.5, 0.5, 0.5)
train_transform = AlbumentationTransforms([       
                                                                
                                      A.HorizontalFlip(p = 0.7),
                                      A.PadIfNeeded(min_height=70, min_width=70, border_mode=4, value=None, mask_value=None, always_apply=False, p=1.0),
                                      A.GaussNoise(var_limit=(10.0, 50.0), mean=0, always_apply=False, p=0.5),
                                      A.RandomCrop(64, 64, always_apply=False, p=1.0),
                                      # A.transforms.ShiftScaleRotate()
                                      A.Normalize(mean=channel_means, std=channel_stdevs),
                                      A.Cutout(num_holes=1, max_h_size=32,max_w_size = 32,p=0.7) 
                                       ])
test_transform = AlbumentationTransforms([A.Normalize(mean=channel_means, std=channel_stdevs)])
train_dataset , test_dataset,classes = TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)

Images already downloaded...


## **DataLoader**

In [0]:
trainloader, testloader = train_test_loader.load(train_dataset,test_dataset,batch_size=640)

## **Used Torchvisions resnet model, but not pre trained. Print the summary**

In [5]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)
summary(model, input_size=(3, 64, 64))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
       BasicBlock-11           [-1, 64, 16, 16]               0
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,

## **Initialise Optimiser,Scheduler, Criterion**

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9 ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, max_lr = 0.2, total_steps=None, epochs=20, steps_per_epoch=1, pct_start=0.5, anneal_strategy='linear', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=10.0,final_div_factor =1)


## **Train Model**

In [7]:

train_test.train_model(model,device,trainloader,testloader,optimizer,criterion,20,scheduler,batch_scheduler=False,best_acc = 0,path = "tinymodel.pt")

  0%|          | 0/121 [00:00<?, ?it/s]

EPOCH: 1 LR: 0.02


Loss=4.554074287414551 Batch_id=120 Accuracy=3.15: 100%|██████████| 121/121 [03:29<00:00,  1.73s/it]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0073, Accuracy: 2039/33000 (6.18%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.04


Loss=4.117725372314453 Batch_id=120 Accuracy=9.51: 100%|██████████| 121/121 [03:29<00:00,  1.73s/it]
  0%|          | 0/121 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0050, Accuracy: 11685/33000 (35.41%)

EPOCH: 19 LR: 0.03799999999999998


Loss=1.1192513704299927 Batch_id=120 Accuracy=70.41: 100%|██████████| 121/121 [03:27<00:00,  1.72s/it]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0050, Accuracy: 11994/33000 (36.35%)

accuracy increased, Saving model....
EPOCH: 20 LR: 0.01999999999999999


Loss=0.9433963298797607 Batch_id=120 Accuracy=76.92: 100%|██████████| 121/121 [03:27<00:00,  1.72s/it]
100%|██████████| 52/52 [00:08<00:00,  6.11it/s]



Test set: Average loss: 0.0049, Accuracy: 12365/33000 (37.47%)

accuracy increased, Saving model....


In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9 ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, max_lr = 0.01, total_steps=None, epochs=20, steps_per_epoch=1, pct_start=0.1, anneal_strategy='linear', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=10.0,final_div_factor =1)


In [9]:

train_test.train_model(model,device,trainloader,testloader,optimizer,criterion,20,scheduler,batch_scheduler=False,best_acc = 0,path = "tinymodel.pt")

  0%|          | 0/121 [00:00<?, ?it/s]

EPOCH: 1 LR: 0.001


Loss=0.750038206577301 Batch_id=120 Accuracy=82.02: 100%|██████████| 121/121 [03:29<00:00,  1.73s/it]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0048, Accuracy: 12875/33000 (39.02%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.010000000000000002


Loss=0.6172533631324768 Batch_id=120 Accuracy=83.01: 100%|██████████| 121/121 [03:28<00:00,  1.72s/it]
  0%|          | 0/121 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0050, Accuracy: 12790/33000 (38.76%)

EPOCH: 19 LR: 0.0014999999999999996


Loss=0.46783557534217834 Batch_id=120 Accuracy=89.47: 100%|██████████| 121/121 [03:27<00:00,  1.72s/it]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0050, Accuracy: 12803/33000 (38.80%)

EPOCH: 20 LR: 0.0009999999999999992


Loss=0.5373174548149109 Batch_id=120 Accuracy=89.96: 100%|██████████| 121/121 [03:27<00:00,  1.72s/it]
100%|██████████| 52/52 [00:08<00:00,  6.12it/s]


Test set: Average loss: 0.0050, Accuracy: 12820/33000 (38.85%)



In [0]:
def train_model(model,device,trainloader,testloader,optimizer,criterion,EPOCHS,scheduler = False,batch_scheduler = False ,best_acc = 0,path = "/content/gdrive/My Drive/API/bestmodel.pt"):
  for epoch in range(EPOCHS):
      print("EPOCH:", epoch+1,'LR:',optimizer.param_groups[0]['lr'])
     
      train_scheduler = False

      if(batch_scheduler):
        train_scheduler = scheduler
      train_loss, train_acc = train_test.train(model, device, trainloader, optimizer, criterion, epoch,train_scheduler)
      
      test_loss , test_acc = train_test.test(model, device, criterion, testloader)
      if(scheduler and not batch_scheduler and not isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau)): 
        scheduler.step()

      elif(scheduler and not batch_scheduler and  isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau)):
        scheduler.step(test_loss[-1])
      
      
      if(test_acc[-1]>best_acc):
        print("accuracy increased, Saving model....")
        best_acc = test_acc[-1]
        torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': test_loss,
              }, path)


In [0]:

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau,StepLr
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=0.9 ,weight_decay = 0.0001,nesterov = True)
scheduler = ReduceLROnPlateau(optimizer, patience = 3,min_lr=1e-7)

In [17]:

train_model(model,device,trainloader,testloader,optimizer,criterion,10,scheduler,batch_scheduler=False,best_acc = 0,path = "tinymodel.pt")

  0%|          | 0/121 [00:00<?, ?it/s]

EPOCH: 1 LR: 0.001


Loss=0.49047529697418213 Batch_id=120 Accuracy=89.97: 100%|██████████| 121/121 [03:27<00:00,  1.71s/it]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0050, Accuracy: 12851/33000 (38.94%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.001


Loss=0.33606788516044617 Batch_id=120 Accuracy=90.19: 100%|██████████| 121/121 [03:26<00:00,  1.71s/it]
  0%|          | 0/121 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0050, Accuracy: 12837/33000 (38.90%)

EPOCH: 9 LR: 0.0001


Loss=0.28567931056022644 Batch_id=120 Accuracy=90.38: 100%|██████████| 121/121 [03:26<00:00,  1.71s/it]
  0%|          | 0/121 [00:00<?, ?it/s]


Test set: Average loss: 0.0050, Accuracy: 12821/33000 (38.85%)

EPOCH: 10 LR: 0.0001


Loss=0.37283238768577576 Batch_id=120 Accuracy=90.48: 100%|██████████| 121/121 [03:26<00:00,  1.71s/it]
100%|██████████| 52/52 [00:08<00:00,  5.99it/s]


Test set: Average loss: 0.0050, Accuracy: 12833/33000 (38.89%)

